In [1]:
%load_ext tclmagic

#  If the prev line doesnt work  install it using the next line in your unix terminal
#  /depot/Python/Python-3.8.0/bin/pip install -U tcl-magic
#  /depot/Python/Python-3.8.0/bin/pip install -U notebook
#  You can configure it to ~/.local/ 

In [2]:
%%tcl 

set SHELLTOOL_LOC "/remote/cad-rep/msip/tools/Shelltools"
lappend auto_path "$SHELLTOOL_LOC/ddr-utils-lay/dev/lib/tcl"
lappend auto_path "/depot/tcl8.6.3/lib"

set RealBin [file dirname [file normalize [info script]] ]
set RealScript [file tail [file normalize [info script]] ]
set PROGRAM_NAME $RealScript
set LOGFILE "[pwd]/log-$PROGRAM_NAME.log"

package require Messaging 1.0
namespace import ::Messaging::*
package require Misc 1.0
namespace import ::Misc::*

package require cmdline
package require try
package require yaml

'0.3.10'

# 1338 --> yml2tcl

The next experiments are related to make a reader of yml for TCL

In [3]:
%%tcl
file exists sample.yml

1

In [4]:
%%tcl

yaml::yaml2dict -file sample.yml

'calibre_verifs 1 cdl_prune_cells {cvcp* cvpp* vflag* vsync} ferel fe_weekly layers {M0 M1 M2 M3 M4 M5 M6 M7 M8 M9 M10 OVERLAP} layers_override {dwc_ddr5phycover_dx4_top_ew {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phycover_cmosx2_top_ew {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phycover_ckx2_top_ew {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phycover_master_top {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phy_utility_blocks {M0 M1 M2 M3 M4 M5 M6 M7 M8 M9 M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1 AP} dwc_ddr5phy_tcoil_ew {M14 M15 M16 MTOP MTOP-1} dwc_ddr5phycover_zcal_top_ew {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phycover_acx2_top_ew {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phydx4_tcoil_ap2c4d_ew {M0 M1 M2 M3 M4 M5 M6 M7 M8 M9 M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1 AP}} layout_tag {D910 Final Release} lef_diff_rel 1.00a_pre3 metal_stack 15M_1X_h_1Xb_v_1Xe_h_1Ya_v_1Yb_h_5Y_vhvhv_2Yy2Z metal_stack_cover 15M_1X_h_1Xb_v_1Xe_h_1Ya_v_1Yb_h_5Y_vhv

In [5]:
%%tcl

set sample_path "/u/alvaro/GitLab/my-personal-projects/experiments/tclyaml_experiment/inputs/sample.tcl"

'/u/alvaro/GitLab/my-personal-projects/experiments/tclyaml_experiment/inputs/sample.tcl'

In [10]:
%%tcl
proc create_dict_from_file {__file} {

    source $__file
    unset __file

    set varlist [lsort [info vars]]

    foreach varname $varlist {
        if {[array exists $varname]} {
            foreach {ark arv} [array get $varname] {
                # iprint "$varname $ark --> $arv"
                dict set legal_release $varname "$ark: " "\{$arv\}"
            }
        } else {
            set varvalue [set $varname]
            dict set legal_release $varname $varvalue
            unset varvalue
        }
        
        
    }
    return $legal_release
}

In [17]:
%%tcl
proc Main {sample_path {file_name "yaml-log.yml"}} {
    set legal_release [create_dict_from_file $sample_path]
    set yaml_format [yaml::dict2yaml $legal_release 4 40]
    nprint [regsub -all {\: \} \{} $yaml_format ": "]
    write_file $file_name [regsub -all {\|-|>| \{|\} } [regsub -all {\} | \{} $yaml_format "\}\n    "] ""] 
}

proc dict2yml {legalRelease_file} {
    return  [yaml::dict2yaml $legal_release 4 80]
}

proc ymlfile2dict {ymlFile} {
    return [yaml::yaml2dict -file $ymlFile]
}

proc clean_yml {ymlResult} {

}

In [68]:
%%tcl
proc Main {sample_path {file_name "yaml-log.yml"}} {
    set legal_release [create_dict_from_file $sample_path]
    set yaml_format [yaml::dict2yaml $legal_release 4 100]

    set joining_values [regsub -all {\: \} \{} $yaml_format ": "]
    set cleaning_values [regsub -all {\} \{|\|-\n    \{|>\n    } $joining_values "\n    " ]
    
    set cleaned_values [regsub -all {\: \{} $cleaning_values ": " ]
    write_file $file_name [regsub -all {\}\n} [regsub -all {\}\n} $cleaned_values "\n" ] "\n" ]
}

proc dict2yml {legalRelease_file} {
    return  [yaml::dict2yaml $legal_release 4 40]
}

proc ymlfile2dict {ymlFile} {
    return [yaml::yaml2dict -file $ymlFile]
}

proc clean_yml {ymlResult} {

}

In [58]:
%%tcl
proc Main {sample_path {file_name "yaml-raw.yml"}} {
    set legal_release [create_dict_from_file $sample_path]
    set yaml_format [yaml::dict2yaml $legal_release 4 80]
    write_file $file_name [$yaml_format]
}

proc dict2yml {legalRelease_file} {
    return  [yaml::dict2yaml $legal_release 4 40]
}

proc ymlfile2dict {ymlFile} {
    return [yaml::yaml2dict -file $ymlFile]
}

proc clean_yml {ymlResult} {

}

In [65]:
%%tcl
Main $sample_path

In [57]:
%%tcl
set array_words "\(layers_override\|supply_pins_override\)"
dict for {key value} [ymlfile2dict "yaml-log.yml" ] {
    nprint "$key --> $value"
    global $key
    if [info exists $key] {
        unset $key
    }
    global $key

    try {
        if {[regexp $array_words $key match]} {
            array set $key $value
            parray $key  
        } else {
            set $key $value
        }
        
    } on error {res opts} {
        
    }

}



calibre_verifs --> 1
cdl_prune_cells --> cvcp* cvpp* vflag* vsync
ferel --> fe_weekly
layers --> M0 M1 M2 M3 M4 M5 M6 M7 M8 M9 M10 OVERLAP
layers_override --> dwc_ddr5phycover_dx4_top_ew {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phycover_cmosx2_top_ew {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phycover_ckx2_top_ew {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phycover_master_top {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phy_utility_blocks {M0 M1 M2 M3 M4 M5 M6 M7 M8 M9 M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1 AP} dwc_ddr5phy_tcoil_ew {M14 M15 M16 MTOP MTOP-1} dwc_ddr5phycover_zcal_top_ew {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phycover_acx2_top_ew {M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1} dwc_ddr5phydx4_tcoil_ap2c4d_ew {M0 M1 M2 M3 M4 M5 M6 M7 M8 M9 M10 M11 M12 M13 M14 M15 M16 MTOP MTOP-1 AP}
layers_override(dwc_ddr5phy_tcoil_ew)           = M14 M15 M16 MTOP MTOP-1
layers_override(dwc_ddr5phy_utility_blocks)     = M0 M1 M2 M3 M4 M5 M6 M7 M8 M9 M10 M1

# Make request to Haashim

Ask him to rewrite and create a new method for this: (including unit test)
```tcl
# Project topcells report
set currDate [clock scan [clock format [clock seconds] -format {%m/%d/%Y}] -format {%m/%d/%Y}]
set tcEDMsgs ""
catch {set tcrDate [exec grep "Execution Date:" $topCellsReport 2> /dev/null]}
if {[info exists tcrDate]} {
    regexp {Execution Date:([\d\/]+)} $tcrDate -> tcrDate
    set tcrDate [clock scan $tcrDate -format {%m/%d/%Y}]
    set diffd [expr {$currDate-$tcrDate}]
    if {[expr {$currDate-$tcrDate}] > 86400} {
        puts "WARNING: PCS Release Topcells report was not created within last 24 hours"
        lappend tcEDMsgs "PCS Release Topcells report was not created within last 24 hours\n"
    }
} else {
    puts "ERROR: Cannot find topcells Execution Date"
    puts "INFO:  Please check topcells report and re-run\r"
    exit
}
catch {set tcrDateRB [exec grep "Execution Date:" $topCellsRB 2> /dev/null]}
if {[info exists tcrDateRB]} {
    regexp {Execution Date:([\d\/]+)} $tcrDateRB -> tcrDateRB
    set tcrDateRB [clock scan $tcrDateRB -format {%m/%d/%Y}]
    set diffd [expr {$currDate-$tcrDateRB}]
    if {[expr {$currDate-$tcrDateRB}] > 86400} {
        puts "WARNING: Release branch Topcells report was not created within last 24 hours"
        lappend tcEDMsgs "Release branch Topcells report was not created within last 24 hours\n"
    }
} else {
    puts "ERROR: Cannot find topcells Execution Date"
    puts "INFO:  Please check topcells report and re-run\r"
    exit
}
```

In [52]:
%%tcl

proc lmap {_var list body} {
    upvar 1 $_var var
    set res {}
    foreach var $list {lappend res [uplevel 1 $body]}
    set res
}

lmap v {1 2 3 a b c} {
    if {![string is entier $v]} continue
    set v
    }

TclError: bad class "entier": must be alnum, alpha, ascii, control, boolean, digit, double, false, graph, integer, list, lower, print, punct, space, true, upper, wideinteger, wordchar, or xdigit

In [67]:
%%tcl
update

TclError: "update" isn't a procedure